In [1]:
# Load the model
import joblib
rf = joblib.load('app/handlers/model.pkl')

In [2]:
# Create the pipeline
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
c = make_pipeline(rf)

In [3]:
# Load the production data
import sklearn
import pandas as pd
import numpy as np
df = pd.read_csv('prod.csv', sep=',')
df
# We chose to remove all attributtes not relevant to the CMU setting.
# `school` is specific to the source data set, and the grades are also
# not relevant (we keep the last grade to know the expected prediction value)
df.drop(columns=['school', 'G1', 'G2'], inplace=True)
df

df['sex'] = np.where(df['sex'] == 'F', 1, 0)
df['address'] = np.where(df['address'] == 'U', 1, 0)
df['famsize'] = np.where(df['famsize'] == 'LE3', 1, 0)
df['Pstatus'] = np.where(df['Pstatus'] == 'T', 1, 0)

for binary_attribute in ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    df[binary_attribute] = np.where(df[binary_attribute] == 'yes', 1, 0)

df


categories = {
    'Mjob': ['teacher', 'health', 'services', 'at_home', 'other'],
    'Fjob': ['teacher', 'health', 'services', 'at_home', 'other'],
    'reason': ['home', 'reputation', 'course', 'other'],
    'guardian': ['mother', 'father', 'other'],
}

for category in categories:
    values = categories[category]
    df[category] = df[category].map(values.index)

df

df['qual_student'] = np.where(df['G3'] >= 15, 1, 0)
df.drop(columns=['G3'], inplace=True)

df


dependent_variable = 'qual_student'
x_raw = df[df.columns.difference([dependent_variable])]
x = x_raw.to_numpy()
y = df[dependent_variable].to_numpy()


In [4]:
# Create the explainer
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(
  x,
  feature_names=x_raw.columns,
  class_names=['not_qualified', 'qualified'],
  discretize_continuous=True
)

In [5]:
# Run the explainer on N random production items
counts = {}
criteria = set()

N = 100

for index in range(N):
    print("——————————————————————————————")
    print(index + 1, "/", N)

    i = np.random.randint(0, x.shape[0])
    row = x[i]

    exp = explainer.explain_instance(row, rf.predict_proba)

    exp_map = exp.as_map()

    for (criterion, weight) in exp.as_list():
      if weight > 0:
        criteria.add(criterion)

    result = exp_map[list(exp_map.keys())[0]]
    for (feature_number, weight) in result:
        feature = x_raw.columns[feature_number]
        if not feature in counts: counts[feature] = 0
        counts[feature] += abs(weight)


——————————————————————————————
1 / 100


/Users/Nicolas/.local/share/virtualenvs/hw45-V0go0g1U/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 1786 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 2436 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 3186 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 4036 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 4986 tasks      | elapsed:    0.9s
[Parallel(n_jobs=7)]: Done 6036 tasks      | elapsed:    1.1s
[Parallel(n_job

——————————————————————————————
2 / 100


[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:    0.4s
[Parallel(n_jobs=7)]: Done 1786 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 2436 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 3186 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 4036 tasks      | elapsed:    0.9s
[Parallel(n_jobs=7)]: Done 4986 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 6036 tasks      | elapsed:    1.2s
[Parallel(n_jobs=7)]: Done 7186 tasks      | elapsed:    1.4s
[Parallel(n_jobs=7)]: Done 8436 tasks      | elapsed:    1.6s
[Parallel(n_jobs=7)]: Done 9786 tasks      | elapsed:    1.9s
[Parallel(n_jobs=7)]: Done 11236 tasks      | elapsed:    2.1s
[Parallel(n_jobs=7)]: Done 11750 out of 11750 | elapsed:    2.2s finished
/Users/Nicolas/.local/share/virtualenvs/hw45-V0go0g1U/lib/py

——————————————————————————————
3 / 100


...

[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:    0.4s
[Parallel(n_jobs=7)]: Done 1786 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 2436 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 3186 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 4036 tasks      | elapsed:    0.8s
[Parallel(n_jobs=7)]: Done 4986 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 6036 tasks      | elapsed:    1.2s
[Parallel(n_jobs=7)]: Done 7186 tasks      | elapsed:    1.4s
[Parallel(n_jobs=7)]: Done 8436 tasks      | elapsed:    1.6s
[Parallel(n_jobs=7)]: Done 9786 tasks      | elapsed:    1.8s
[Parallel(n_jobs=7)]: Done 11236 tasks      | elapsed:    2.0s
[Parallel(n_jobs=7)]: Done 11750 out of 11750 | elapsed:    2.1s finished
/Users/Nicolas/.local/share/virtualenvs/hw45-V0go0g1U/lib/py

——————————————————————————————
99 / 100


[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:    0.4s
[Parallel(n_jobs=7)]: Done 1786 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 2436 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 3186 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 4036 tasks      | elapsed:    0.8s
[Parallel(n_jobs=7)]: Done 4986 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 6036 tasks      | elapsed:    1.2s
[Parallel(n_jobs=7)]: Done 7186 tasks      | elapsed:    1.4s
[Parallel(n_jobs=7)]: Done 8436 tasks      | elapsed:    1.6s
[Parallel(n_jobs=7)]: Done 9786 tasks      | elapsed:    1.8s
[Parallel(n_jobs=7)]: Done 11236 tasks      | elapsed:    2.0s
[Parallel(n_jobs=7)]: Done 11750 out of 11750 | elapsed:    2.1s finished
/Users/Nicolas/.local/share/virtualenvs/hw45-V0go0g1U/lib/py

——————————————————————————————
100 / 100


[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:    0.4s
[Parallel(n_jobs=7)]: Done 1786 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 2436 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 3186 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 4036 tasks      | elapsed:    0.8s
[Parallel(n_jobs=7)]: Done 4986 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 6036 tasks      | elapsed:    1.1s
[Parallel(n_jobs=7)]: Done 7186 tasks      | elapsed:    1.3s
[Parallel(n_jobs=7)]: Done 8436 tasks      | elapsed:    1.5s
[Parallel(n_jobs=7)]: Done 9786 tasks      | elapsed:    1.8s
[Parallel(n_jobs=7)]: Done 11236 tasks      | elapsed:    2.0s
[Parallel(n_jobs=7)]: Done 11750 out of 11750 | elapsed:    2.1s finished


## Influential features

In [10]:
sorted_counts = dict(sorted(counts.items(), key=lambda item: -item[1]))
sorted_counts

{'Medu': 5.481959086353252,
 'failures': 5.453884906187786,
 'Mjob': 5.233616210701514,
 'schoolsup': 2.8721988421246585,
 'Dalc': 2.778989428503357,
 'Fjob': 2.6356892995604184,
 'Walc': 2.4585218379106077,
 'absences': 2.1306026225044645,
 'sex': 1.7736518761028295,
 'age': 1.6118524023667569,
 'goout': 0.532462522620434,
 'studytime': 0.5145284614832921,
 'freetime': 0.300679323224309,
 'reason': 0.15741780428934876,
 'romantic': 0.07167684809446624,
 'famrel': 0.014283853099889636}

## Ideal values

In [8]:
criteria

{'2.00 < Medu <= 3.00',
 '3.00 < Medu <= 4.00',
 '4.00 < famrel <= 5.00',
 'Dalc <= 1.00',
 'Fjob <= 2.00',
 'Mjob <= 2.00',
 'Walc <= 1.00',
 'absences <= 2.00',
 'age <= 16.00',
 'failures <= 0.00',
 'freetime > 4.00',
 'goout <= 2.00',
 'reason <= 0.00',
 'romantic <= 0.00',
 'schoolsup <= 0.00',
 'sex <= 0.00',
 'studytime > 2.00'}